# Predicter Neural Network 

### Importing the libraries

In [ ]:
import os
os.chdir(os.path.expanduser("/Users/barkinkaradeniz/Projects/ChatGPTStockPredicter copy/"))

from src.mongo.pymongo_get_database import get_database

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from catboost import CatBoostClassifier

from datetime import datetime  

In [ ]:
tf.__version__

## Data Preprocessing

### Importing the dataset

In [ ]:
dbname = get_database()
averages = dbname["AlphabetSentimentAveragesUpdated"]
changes = dbname["AlphabetDailyChange"]

In [ ]:
X = np.zeros((337, 10))
X[:, 0] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-07"}}, {"date" : {"$lte" : "2023-07-10"}}]})])
X[:, 1] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-08"}}, {"date" : {"$lte" : "2023-07-11"}}]})])
X[:, 2] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-09"}}, {"date" : {"$lte" : "2023-07-12"}}]})])
X[:, 3] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-10"}}, {"date" : {"$lte" : "2023-07-13"}}]})])
X[:, 4] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-11"}}, {"date" : {"$lte" : "2023-07-14"}}]})])
X[:, 5] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-14"}}, {"date" : {"$lte" : "2023-07-17"}}]})])
X[:, 6] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-15"}}, {"date" : {"$lte" : "2023-07-18"}}]})])
X[:, 7] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-16"}}, {"date" : {"$lte" : "2023-07-19"}}]})])
X[:, 8] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-17"}}, {"date" : {"$lte" : "2023-07-20"}}]})])
X[:, 9] = np.array([average["sentimentAverage"] for average in averages.find({"$and" : [{"date": {"$gte" : "2022-03-18"}}, {"date" : {"$lte" : "2023-07-21"}}]})])

# X[:, 10] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-07"}}, {"date" : {"$lte" : "2023-07-10"}}]})])
# X[:, 11] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-08"}}, {"date" : {"$lte" : "2023-07-11"}}]})])
# X[:, 12] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-09"}}, {"date" : {"$lte" : "2023-07-12"}}]})])
# X[:, 13] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-10"}}, {"date" : {"$lte" : "2023-07-13"}}]})])
# X[:, 14] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-11"}}, {"date" : {"$lte" : "2023-07-14"}}]})])
# X[:, 15] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-14"}}, {"date" : {"$lte" : "2023-07-17"}}]})])
# X[:, 16] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-15"}}, {"date" : {"$lte" : "2023-07-18"}}]})])
# X[:, 17] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-16"}}, {"date" : {"$lte" : "2023-07-19"}}]})])
# X[:, 18] = np.array([change["change"] for change in changes.find({"$and" : [{"date": {"$gte" : "2022-03-17"}}, {"date" : {"$lte" : "2023-07-20"}}]})])

y = np.array([change["change"] for change in changes.find({"date": {"$gte" : "2022-03-18"}})])


### Splitting the dataset into the Training set and Test set

In [ ]:
# from sklearn.model_selection import cross_val_score


# model = CatBoostClassifier(
#     iterations=200,
#     learning_rate=0.02,
#     depth=12,
#     eval_metric="Accuracy",
#     random_seed = 2112,
#     bagging_temperature = 0.2,
#     od_type="Iter",
#     od_wait=100,
#     verbose=0,
#     task_type="CPU"
# )


# scores = cross_val_score(model, X, y, cv=5)

# scores

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.02,
    depth=12,
    eval_metric="Accuracy",
    random_seed = 2112,
    bagging_temperature = 0.2,
    od_type="Iter",
    od_wait=100,
    verbose=1,
)

model.fit(X_train, y_train, eval_set=(X_test, y_test))

## Building the ANN

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))

In [ ]:
# ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 80)

In [ ]:
y_pred = ann.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
y_pred = (y_pred > 0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)